In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
sns.set()
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot

In [3]:
#import file
#remember to change '\' to '/'
os.chdir('/Users/kane/Desktop/NTU BCG/Year 2 Sem 2/BC2407 Analytics II/BC2407 Course Materials/S2 Review of Basic Analytics') #set path
flatprices = pd.read_csv('resale-flat-prices-2019.csv')

# 1. Creating derived variable (remaining_lease_years)

In [6]:
def remainingleaseyears(s):
    string = s['remaining_lease']
    if ('month' in string):
        years = string[0:2]
        if (string[9] == '0' and string[10] == ' '): #means 0 months
            months = 0
        elif (string[9] == '0' and string[10] != ' '): #means not 0 months, but month between 1-9 months
            months = string[10]
        else:
            months = string[9:11] #for 10/11/12 months
        leaseyears = float(years) + (float(months)/12)
    else: #only contains year not month
        years = string[0:2]
        leaseyears = float(years)
    return leaseyears
flatprices['remaining_lease_years'] = flatprices.apply(remainingleaseyears,axis=1)

In [7]:
categoricalcols = ['town', 'flat_type', 'storey_range', 'flat_model']
flatprices[categoricalcols] = flatprices[categoricalcols].astype('category')

In [8]:
flatprices

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,remaining_lease_years
0,2019-01,ANG MO KIO,3 ROOM,330,ANG MO KIO AVE 1,01 TO 03,68.0,New Generation,1981,61 years 01 month,270000.0,61.083333
1,2019-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,04 TO 06,73.0,New Generation,1976,56 years 04 months,295000.0,56.333333
2,2019-01,ANG MO KIO,3 ROOM,225,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1978,58 years 01 month,270000.0,58.083333
3,2019-01,ANG MO KIO,3 ROOM,225,ANG MO KIO AVE 1,01 TO 03,67.0,New Generation,1978,58 years,230000.0,58.000000
4,2019-01,ANG MO KIO,3 ROOM,333,ANG MO KIO AVE 1,01 TO 03,68.0,New Generation,1981,61 years,262500.0,61.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
22199,2019-12,YISHUN,EXECUTIVE,326,YISHUN RING RD,04 TO 06,142.0,Apartment,1988,67 years 07 months,580000.0,67.583333
22200,2019-12,YISHUN,EXECUTIVE,326,YISHUN RING RD,07 TO 09,146.0,Maisonette,1988,67 years 07 months,565000.0,67.583333
22201,2019-12,YISHUN,EXECUTIVE,608,YISHUN ST 61,01 TO 03,164.0,Apartment,1992,71 years 06 months,633000.0,71.500000
22202,2019-12,YISHUN,EXECUTIVE,604,YISHUN ST 61,10 TO 12,164.0,Apartment,1992,71 years 06 months,788888.0,71.500000


# 2. Set Baseline Reference level for "town" to "Yishun"

In [9]:
m1 = smf.ols('resale_price ~ floor_area_sqm + remaining_lease_years + storey_range + C(town, Treatment(reference = "YISHUN"))', 
             flatprices).fit()

In [11]:
m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           resale_price   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     2734.
Date:                Thu, 19 Jan 2023   Prob (F-statistic):               0.00
Time:                        11:58:11   Log-Likelihood:            -2.7628e+05
No. Observations:               22204   AIC:                         5.526e+05
Df Residuals:                   22160   BIC:                         5.530e+05
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                 -4.538e+05   3877.965   -117.020      0.000   -4.61e+05   -4.46e+05
storey_range[T.04 TO 06]                                   1.377e+04   1288.149     10.688      0.000    1.12e+04    1.63e+04
storey_range[T.07 TO 09]                                   2.684e+04   1322.825     20.293      0.000    2.43e+04    2.94e+04
storey_range[T.10 TO 12]                                   3.778e+04   1356.846     27.840      0.000    3.51e+04    4.04e+04
storey_range[T.13 TO 15]                                   4.559e+04   1714.217     26.598      0.000    4.22e+04     4.9e+04
storey_range[T.16 TO 18]                                   6.064e+04   2246.439     26.995      0.000    5.62e+04     6.5e+04
storey_range[T.19 TO 21]                                   9.509e+04   3265.439     29.121      0.000    8.87e+04    1.01e+05
storey_range[T.22 TO 24]                                   1.034e+05   3688.099     28.023      0.000    9.61e+04    1.11e+05
storey_range[T.25 TO 27]                                   1.269e+05   5285.540     24.013      0.000    1.17e+05    1.37e+05
storey_range[T.28 TO 30]                                   1.772e+05   6468.563     27.393      0.000    1.65e+05     1.9e+05
storey_range[T.31 TO 33]                                   1.919e+05   1.05e+04     18.192      0.000    1.71e+05    2.13e+05
storey_range[T.34 TO 36]                                   1.923e+05   1.15e+04     16.661      0.000     1.7e+05    2.15e+05
storey_range[T.37 TO 39]                                   2.403e+05   1.07e+04     22.470      0.000    2.19e+05    2.61e+05
storey_range[T.40 TO 42]                                   2.826e+05    1.5e+04     18.803      0.000    2.53e+05    3.12e+05
storey_range[T.43 TO 45]                                   3.529e+05   4.37e+04      8.081      0.000    2.67e+05    4.39e+05
storey_range[T.46 TO 48]                                   3.556e+05   3.11e+04     11.446      0.000    2.95e+05    4.17e+05
storey_range[T.49 TO 51]                                   4.137e+05   6.16e+04      6.719      0.000    2.93e+05    5.34e+05
C(town, Treatment(reference="YISHUN"))[T.ANG MO KIO]       1.161e+05   2535.640     45.792      0.000    1.11e+05    1.21e+05
C(town, Treatment(reference="YISHUN"))[T.BEDOK]            1.105e+05   2388.875     46.261      0.000    1.06e+05    1.15e+05
C(town, Treatment(reference="YISHUN"))[T.BISHAN]           2.173e+05   3374.027     64.408      0.000    2.11e+05    2.24e+05
C(town, Treatment(reference="YISHUN"))[T.BUKIT BATOK]      2.496e+04   2558.393      9.755      0.000    1.99e+04       3e+04
C(town, Treatment(reference="YISHUN"))[T.BUKIT MERAH]      

# Activity 2

# LogReg Model to predict default status 

In [13]:
default = pd.read_csv('default.csv')

In [14]:
#convert relevant fields to categorical
categoricalcols = ['Default', 'Gender']
default[categoricalcols] = default[categoricalcols].astype('category')

#convert relevant fields to categorical
categoricalcols = ['Default']
default[categoricalcols] = default[categoricalcols].astype('category')

In [15]:
X = data_final.loc[:, data_final.columns != 'Default'] #predictors
y = data_final.loc[:, data_final.columns == 'Default'] #response variable

NameError: name 'data_final' is not defined